# Demonstrate the MMCController ophyd support

In [1]:
%run -i xy_positioner

In [2]:
stage = TwoD_XY_StagePositioner("", name="xy_stage")
print(f"{stage = }")

stage = TwoD_XY_StagePositioner(prefix='', name='xy_stage', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['x', 'x.readback', 'x.setpoint', 'y', 'y.readback', 'y.setpoint', 'pair'], configuration_attrs=['x', 'y'], concurrent=True)


In [3]:
print(f"{stage.forward((8.2, 3.14159)) = }")
print(f"{stage.inverse((-.5, -.23)) = }")

print(f"{stage.inverse((25, 6.24)) = }")

stage.forward((8.2, 3.14159)) = TwoD_XY_StagePositionerRealPos(pair=TwoD_XY_StagePositionerPseudoPos(x=8.2, y=3.14159))
stage.inverse((-.5, -.23)) = TwoD_XY_StagePositionerPseudoPos(x=-0.5, y=-0.23)
stage.inverse((25, 6.24)) = TwoD_XY_StagePositionerPseudoPos(x=25, y=6.24)


In [4]:
st = stage.move((10,1))
print(f"{st = }")
st.wait()
print(f"{stage = }")
print(f"{stage.pair.mmc.xy = }")
stage.read()

st = MoveStatus(done=True, pos=xy_stage, elapsed=1.0, success=True, settle_time=0.0)
stage = TwoD_XY_StagePositioner(prefix='', name='xy_stage', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='computed', read_attrs=['x', 'x.readback', 'x.setpoint', 'y', 'y.readback', 'y.setpoint', 'pair'], configuration_attrs=['x', 'y'], concurrent=True)
stage.pair.mmc.xy = TwoD_XY_StagePositionerPseudoPos(x=10, y=1)


OrderedDict([('xy_stage_x', {'value': 10, 'timestamp': 1620097276.1744738}),
             ('xy_stage_x_setpoint',
              {'value': 10, 'timestamp': 1620097276.1744738}),
             ('xy_stage_y', {'value': 1, 'timestamp': 1620097276.1744738}),
             ('xy_stage_y_setpoint',
              {'value': 1, 'timestamp': 1620097276.1744738}),
             ('xy_stage_pair',
              {'value': TwoD_XY_StagePositionerPseudoPos(x=10, y=1),
               'timestamp': 1620097276.1755083})])

In [5]:
stage.move((-1.234, 54.321))
stage.read()

OrderedDict([('xy_stage_x',
              {'value': -1.234, 'timestamp': 1620097276.1744738}),
             ('xy_stage_x_setpoint',
              {'value': -1.234, 'timestamp': 1620097276.1744738}),
             ('xy_stage_y',
              {'value': 54.321, 'timestamp': 1620097276.1744738}),
             ('xy_stage_y_setpoint',
              {'value': 54.321, 'timestamp': 1620097276.1744738}),
             ('xy_stage_pair',
              {'value': TwoD_XY_StagePositionerPseudoPos(x=-1.234, y=54.321),
               'timestamp': 1620097276.1755083})])